# Python文本解释器

本项目将创建以下模块，每个模块的作用简介如下：
1. util.py：实现文本块生成器把纯文本分成一个一个的文本块，以便接下来对每一个文本块进行解析
2. handlers.py：为文本块打上合适的 HTML 标记
3. rules.py：设计一定的规则来判断每个文本块交给处理程序将要加什么标记
4. markup.py：对整个文本进行解析的程序

## 1. 文本块生成器

首先我们需要有一个文本块生成器把纯文本分成一个一个的文本块（前后以空行分割），以便接下来对每一个文本块进行解析，util.py 代码如下：

In [1]:
#!/usr/bin/python
# encoding: utf-8

def lines(file):
    """
    生成器,在文本最后加一空行
    """
    for line in file: yield line
    yield '\n'

def blocks(file):
    """
    生成器,生成单独的文本块
    """
    block = []
    for line in lines(file):
        # 非空行,加到当前的block中
        if line.strip():
            block.append(line)
        # 遇到空行，将当前保存的block连接起来，然后block清空
        elif block:
            yield ''.join(block).strip()
            block = []

### strip()介绍
strip() 函数可以去除一个字符串前后的空格以及换行符，如果在strip()函数添加不同的参数，如strip("me")，则可以去除字符串前后的"me"字符。

```
>>> s = "  This is me\n"
>>> s = s.strip()
>>> s
'This is me'
>>> s = s.strip("me")
>>> s
'This is '
```
yield()会返回一个生成器(generator)。

## 2.  处理程序

假设我们已经知道一个文本块是 title/paragraph/heading/list，我们通过 handlers.py 给他们打上合适的 HTML 标记。代码如下。

其中， callable() 函数能够检查一个函数是否能够被调用。如果能够被调用返回True。

gerattr()函数则是返回一个对象的属性值。举例来说，getattr(x, 'foo', None) 就相当于是 x.foo，而如果没有这个属性值foo，则返回我们设定的默认值None。

In [2]:
#!/usr/bin/python
# encoding: utf-8

class Handler:
    """
    处理程序父类
    """
    
    # *arg 的作用是将多个参数保存为元组
    # name是某种标签，prefix是为了选择sub_*、start_*、end_*方法
    # 如果名为 prefix_name 处理方法存在，callback函数就返回这个方法的调用结果
    def callback(self, prefix, name, *args):
        method = getattr(self, prefix + name, None)
        if callable(method): return method(*args)
    
    # 调用 start_name 函数的通用方法 
    def start(self, name):
        self.callback('start_', name)
    
    # 调用 end_name 函数的通用方法
    def end(self, name):
        self.callback('end_', name)
    
    # 返回一个函数：
    # 这个函数返回sub_name(match)
    # 如果sub_name 不存在，返回match.group(0)
    def sub(self, name):
        def substitution(match):
            result = self.callback('sub_', name, match)
            if result is None: result = match.group(0)
            return result
        return substitution

class HTMLRenderer(Handler):
    """
    HTML处理程序,给文本块加相应的HTML标记
    """
    def start_document(self):
        print('<html><head><title>ShiYanLou</title></head><body>')

    def end_document(self):
        print('</body></html>')

    def start_paragraph(self):
        print('<p style="color: #444;">')

    def end_paragraph(self):
        print('</p>')

    def start_heading(self):
        print('<h2 style="color: #68BE5D;">')

    def end_heading(self):
        print('</h2>')

    def start_list(self):
        print('<ul style="color: #363736;">')

    def end_list(self):
        print('</ul>')

    def start_listitem(self):
        print('<li>')

    def end_listitem(self):
        print('</li>')

    def start_title(self):
        print('<h1 style="color: #1ABC9C;">')

    def end_title(self):
        print('</h1>')

    def sub_emphasis(self, match):
        return('<em>%s</em>' % match.group(1))

    def sub_url(self, match):
        return('<a target="_blank" style="text-decoration: none;color: #BC1A4B;" href="%s">%s</a>' % (match.group(1), match.group(1)))

    def sub_mail(self, match):
        return('<a style="text-decoration: none;color: #BC1A4B;" href="mailto:%s">%s</a>' % (match.group(1), match.group(1)))

    def feed(self, data):
        print(data)

##  3. 规则

有了处理程序和文本块生成器，接下来就需要一定的规则来判断每个文本块交给处理程序将要加什么标记，rules.py 代码如下：

In [3]:
#!/usr/bin/python
# encoding: utf-8

class Rule:
    """
    规则父类
    """
    def action(self, block, handler):
        """
        加标记
        """
        handler.start(self.type)
        handler.feed(block)
        handler.end(self.type)
        return True

class HeadingRule(Rule):
    """
    一号标题规则
    """
    type = 'heading'
    def condition(self, block):
        """
        判断文本块是否符合规则
        """
        return not '\n' in block and len(block) <= 70 and not block[-1] == ':'

class TitleRule(HeadingRule):
    """
    二号标题规则
    """
    type = 'title'
    first = True
    
    # 判断文本是否符合规则
    def condition(self, block):
        if not self.first: return False
        self.first = False
        return HeadingRule.condition(self, block)

class ListItemRule(Rule):
    """
    列表项规则
    """
    type = 'listitem'
    def condition(self, block):
        return block[0] == '-'

    def action(self, block, handler):
        handler.start(self.type)
        handler.feed(block[1:].strip())
        handler.end(self.type)
        return True

class ListRule(ListItemRule):
    """
    列表规则
    """
    type = 'list'
    inside = False
    def condition(self, block):
        return True

    def action(self, block, handler):
        if not self.inside and ListItemRule.condition(self, block):
            handler.start(self.type)
            self.inside = True
        elif self.inside and not ListItemRule.condition(self, block):
            handler.end(self.type)
            self.inside = False
        return False

class ParagraphRule(Rule):
    """
    段落规则
    """
    type = 'paragraph'

    def condition(self, block):
        return True

## 4.  解析

当我们知道每一个文本块进行怎么样的处理，交给谁去处理之后，我们就可以对整个文本进行解析了，markup.py 代码如下：

In [ ]:
#!/usr/bin/python
# encoding: utf-8

import sys, re
from handlers import *
from util import *
from rules import *

class Parser:
    """
    解析器父类
    """
    def __init__(self, handler):
        self.handler = handler
        self.rules = []
        self.filters = []

    def addRule(self, rule):
        """
        添加规则
        """
        self.rules.append(rule)

    def addFilter(self, pattern, name):
        """
        添加过滤器
        """
        def filter(block, handler):
            return re.sub(pattern, handler.sub(name), block)
        self.filters.append(filter)

    def parse(self, file):
        """
        解析
        """
        self.handler.start('document')
        for block in blocks(file):
            # 使用所有的过滤器 和 规则来处理，只有一个匹配到的会执行
            for filter in self.filters:
                block = filter(block, self.handler)
            for rule in self.rules:
                if rule.condition(block):
                    last = rule.action(block, self.handler)
                    if last: break  # 若前面的规则成功处理，则不考虑后面的规则
        self.handler.end('document')

class BasicTextParser(Parser):
    """
    纯文本解析器
    """
    def __init__(self, handler):
        # 初始化解析器：handler, rules=[], filters=[]
        Parser.__init__(self, handler)
        
        # 为解析器添加规则
        self.addRule(ListRule())
        self.addRule(ListItemRule())
        self.addRule(TitleRule())
        self.addRule(HeadingRule())
        self.addRule(ParagraphRule())
        
        # 为解析器添加过滤器
        self.addFilter(r'\*(.+?)\*', 'emphasis')
        self.addFilter(r'(http://[\.a-zA-Z/]+)', 'url')
        self.addFilter(r'([\.a-zA-Z]+@[\.a-zA-Z]+[a-zA-Z]+)', 'mail')

"""
运行程序
"""
handler = HTMLRenderer()
parser = BasicTextParser(handler)
parser.parse(sys.stdin)

## re模块介绍

re模块为python的正则表达式模块。re.sub函数用于替换原字符串中匹配的部分。

>re.sub(pattern, repl, string, count=0, flags=0)

> * pattern：表示正则表达式中的模式字符串；

> * repl：被替换的字符串（既可以是字符串，也可以是函数，此处是函数）；

> * string：要被处理的，要被替换的字符串；

> * count：匹配的次数, 默认是全部替换

> 返回替换后的字符串

示例如下：

### 1. repl 为字符串

In [10]:
import re
# 默认全替换
newstr = re.sub(r"boy|girl","human","boy, girl")
print(newstr)
# 只替换一次
newstr = re.sub(r"boy|girl","human","boy, girl",count=1)
print(newstr)

human, human
human, girl


### 2. repl 为函数

In [9]:
# 将字符串中的数字增加 111
import re;

def pythonReSubDemo():
    """
        demo Pyton re.sub
    """
    inputStr = "hello 123 world 456";

    def _add111(matched):
        intStr = matched.group("number"); #123
        intValue = int(intStr);
        addedValue = intValue + 111; #234
        addedValueStr = str(addedValue);
        return addedValueStr;

    #?P<value>的意思就是命名一个名字为value的组，匹配规则符合后面的/d+
    replacedStr = re.sub("(?P<number>\d+)", _add111, inputStr);
    print("replacedStr=",replacedStr) #hello 234 world 567


pythonReSubDemo();

replacedStr= hello 234 world 567
